In [2]:
import time
import random
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import urllib3

# SSL 경고 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 최대 재시도 횟수
MAX_RETRIES = 3

# URL 설정
url = "https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0007901371"

# Chrome 옵션 설정
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors=yes')

# 헤더 설정 (requests 용)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

for attempt in range(MAX_RETRIES):
    try:
        # 랜덤 대기 시간 추가
        time.sleep(random.uniform(2, 5))

        # requests로 페이지 내용 가져오기 (SSL 검증 비활성화)
        response = requests.get(url, headers=headers, timeout=30, verify=False)
        
        # 응답 상태 확인
        if response.status_code != 200:
            print(f"HTTP 상태 코드: {response.status_code}")
            raise Exception("페이지 로드 실패 (requests)")
        
        # Selenium 실행
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.set_page_load_timeout(60)
        
        # Selenium으로 페이지 열기
        driver.get(url)
        wait = WebDriverWait(driver, 30)

        # Selenium에서 동적 요소 대기
        company_info_element = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "td[headers='row01']"))
        )
        
        # Selenium으로 데이터 크롤링
        company_info_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row01']").text
        representative_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row02']").text

        # 결과 출력
        print("기업명:", company_info_name)
        print("대표자명:", representative_name)

        # 성공하면 반복문 종료
        break

    except Exception as e:
        print(f"시도 {attempt + 1}/{MAX_RETRIES} 실패: {str(e)}")
        if attempt < MAX_RETRIES - 1:
            time.sleep(random.uniform(1, 10))  # 대기 후 재시도
        continue

    finally:
        if 'driver' in locals():
            driver.quit()

시도 1/3 실패: Could not reach host. Are you offline?
시도 2/3 실패: Could not reach host. Are you offline?
시도 3/3 실패: Could not reach host. Are you offline?


In [17]:
!pip install chromedriver-autoinstaller --trusted-host pypi.org --trusted-host files.pythonhosted.org --no-cache-dir

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import time
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# Chrome options
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver path
chromedriver_autoinstaller.install()

# Login credentials
username = "kimle125"  # Replace with your username
password = "Getajob8282##"  # Replace with your password

try:
    # Initialize WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    # Open the login page
    driver.get("https://sminfo.mss.go.kr/cm/sv/CSV001R0.do")
    
    # Wait for the login form to load
    wait = WebDriverWait(driver, 10)
    
    # Locate the username and password fields
    username_field = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/div[1]')))  # Replace with the actual ID of the username field
    password_field = driver.find_element(By.XPATH, '//*[@id="login_password"]')  # Replace with the actual ID of the password field
    
    # Enter login credentials
    username_field.send_keys(username)
    password_field.send_keys(password)
    
    # Submit the form
    password_field.send_keys(Keys.RETURN)
    
    # Wait for login to complete
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button')))  # Replace with an element that appears after login
    
    # Navigate to the target page or perform actions after login
    driver.get("https://sminfo.mss.go.kr/gc/sf/GSF002R0.print")  # Replace with the actual target URL
    
    # Crawl data from the page
    company_info_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row01']").text
    representative_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row02']").text

    # 결과 출력
    print("기업명:", company_info_name)
    print("대표자명:", representative_name)

    
finally:
    # Close the browser
    if 'driver' in locals():
        driver.quit()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [1]:
from selenium import webdriver

# ChromeDriver 경로 설정
driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"  # 사용자 환경에 맞게 경로 수정
driver = webdriver.Chrome(executable_path=driver_path)

# 로그인 페이지로 이동
login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
driver.get(login_url)

# 입력 필드 찾기
username_field = driver.find_element("id", "login_id")  
password_field = driver.find_element("id", "login_password")

# 로그인 정보 입력
username_field.send_keys("kimble125")
password_field.send_keys("Getajob8282##")

# 로그인 버튼 클릭
login_button = driver.find_element("xpath", '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button') # 버튼 속성에 맞게 변경
login_button.click()

# 원하는 페이지로 이동
driver.get("https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0000078229")

# 페이지 소스 가져오기
page_source = driver.page_source

# Crawl data from the page
company_info_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row01']").text
representative_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row02']").text

# 결과 출력
print("기업명:", company_info_name)
print("대표자명:", representative_name)

TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# ChromeDriver 경로 설정
driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"  # 사용자 환경에 맞게 경로 수정
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# 로그인 페이지로 이동
login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
driver.get(login_url)

# 입력 필드 찾기
username_field = driver.find_element(By.ID, "login_id")  
password_field = driver.find_element(By.ID, "login_password")

# 로그인 정보 입력
username_field.send_keys("kimble125")
password_field.send_keys("Getajob8282##")

# 로그인 버튼 클릭
login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button') # 버튼 속성에 맞게 변경
login_button.click()

# 원하는 페이지로 이동
driver.get("https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0000078229")

# 페이지 소스 가져오기
page_source = driver.page_source

# Crawl data from the page
company_info_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row01']").text
representative_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row02']").text

# 결과 출력
print("기업명:", company_info_name)
print("대표자명:", representative_name)

UnexpectedAlertPresentException: Alert Text: 아이디 또는 비밀번호를 잘못 입력하셨습니다. 
[아이디 찾기],[비밀번호 재발급]를 통해 확인하시기 바랍니다. 

5회 이상 입력 오류 시 로그인 제한 됩니다. (현재 오류횟수 1)
Message: unexpected alert open: {Alert text : 아이디 또는 비밀번호를 잘못 입력하셨습니다. 
[아이디 찾기],[비밀번호 재발급]를 통해 확인하시기 바랍니다. 

5회 이상 입력 오류 시 로그인 제한 됩니다. (현재 오류횟수 1)}
  (Session info: chrome=131.0.6778.109)
Stacktrace:
0   chromedriver                        0x00000001051bbb08 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001051b4358 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000104c1c4b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000104c99e3c cxxbridge1$string$len + 603664
4   chromedriver                        0x0000000104c550b0 cxxbridge1$string$len + 321668
5   chromedriver                        0x0000000104c55d00 cxxbridge1$string$len + 324820
6   chromedriver                        0x0000000105186e4c cxxbridge1$str$ptr + 3435328
7   chromedriver             

In [12]:
import requests

# 로그인 URL과 대상 페이지 URL 설정
login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
target_url = "https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0000078229"

# 세션 생성
session = requests.Session()

# 로그인에 필요한 데이터 (폼 데이터)
login_data = {
    "username": "kimble125",
    "password": "Getajob8282##",
    # 필요하면 추가 데이터 (예: CSRF 토큰)
    # "csrf_token": "your_csrf_token"
}

# 로그인 요청
response = session.post(login_url, data=login_data)

# 로그인 성공 여부 확인
if response.status_code == 200 and "Welcome" in response.text:  # 적절한 확인 조건 설정
    print("로그인 성공!")
else:
    print("로그인 실패!")

# 로그인 후 대상 페이지 크롤링
response = session.get(target_url)
print(response.text)  # HTML 출력 또는 원하는 데이터 추출


SSLError: HTTPSConnectionPool(host='sminfo.mss.go.kr', port=443): Max retries exceeded with url: /cm/sv/CSV001R0.do (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))

In [10]:
!pip install beautifulsoup4 --trusted-host pypi.org --trusted-host files.pythonhosted.org

Defaulting to user installation because normal site-packages is not writeable


In [11]:
import requests
from bs4 import BeautifulSoup

# 로그인 URL
login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"

# 세션 생성
session = requests.Session()

# 로그인 페이지 요청
login_page = session.get(login_url)

# HTML 파싱 및 CSRF 토큰 추출
soup = BeautifulSoup(login_page.text, "html.parser")
csrf_token = soup.find("input", {"name": "csrf_token"})["value"]

# 확인
print("CSRF 토큰:", csrf_token)


SSLError: HTTPSConnectionPool(host='sminfo.mss.go.kr', port=443): Max retries exceeded with url: /cm/sv/CSV001R0.do (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# ChromeDriver 경로 설정
driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"  # 사용자 환경에 맞게 경로 수정
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# 로그인 페이지로 이동
login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
driver.get(login_url)

# 입력 필드 찾기
username_field = driver.find_element(By.ID, "login_id")  
password_field = driver.find_element(By.ID, "login_password")

# 로그인 정보 입력(임시 비밀번호)
username_field.send_keys("hoykim125")
password_field.send_keys("hoykim125!!")

# 로그인 버튼 클릭
login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button') # 버튼 속성에 맞게 변경
login_button.click()

# 원하는 페이지로 이동
driver.get("https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0000078229")

# 페이지 소스 가져오기
page_source = driver.page_source

# 해당 페이지에서 시험 데이터 크롤링
company_info_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row01']").text
representative_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row02']").text

# 결과 출력
print("기업명:", company_info_name)
print("대표자명:", representative_name)